# Examen Math - Caculli Giorgio LA196672

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
%matplotlib inline

# Lecture du fichier de données

In [2]:
PATH = "abalone_clean.csv"
abalone = pd.read_csv(PATH)

FileNotFoundError: [Errno 2] No such file or directory: 'abalone_clean.csv'

# Analyse des données

On affiche les données stockés dans le fichier CSV

In [ ]:
abalone.shape

In [ ]:
abalone.info()

In [ ]:
abalone.head()

In [ ]:
abalone.describe()

# Déterminer les meilleurs paramètres ou features

On initialise les "features" avec lesquelles on va travailler

In [ ]:
features = [' Longest Shell measurement', 'Diameter', 'Height', 'Weight of whole', 'Weight of meat', 'Shell weight']

On determine que "Age" est la valeur avec laquelle on souhaite travailler

In [ ]:
name_target = 'Age'

Etant donné que le "Sexe" est marqué par une lettre on le convertira en nombres pour l'analyser

In [ ]:
sex_dummies = pd.get_dummies(abalone['Sex'], prefix='Sex_')

abalone = pd.concat([abalone, sex_dummies], axis=1)

On rempli la colonne "Age" avec des valeurs, ici, le nombre de rings + 1.5

In [ ]:
abalone[name_target] = abalone['Rings'].apply(lambda age: age + 1.5)

On mélange les informations pour pouvoir en tirer des tranches de données que l'on analysera

In [ ]:
indices = np.random.permutation(abalone.index)
length_data = abalone.shape[0]
limit_train = int(length_data * 0.80)
train = abalone.iloc[:limit_train]
test = abalone.iloc[limit_train:]

On initialise un vecteur contenant les différents types de sexes rencontrés, ici M, F et I.

In [ ]:
sexes = [sex for sex in abalone.columns if sex.startswith('Sex_')]
sexes

On crée les valeurs de y

In [ ]:
target = train[name_target]

On ajoute aux features à analyser les différents sexes sous formes de 0 ou 1

In [ ]:
all_features = features + sexes

On affiche le nouveau tableau avec les différents sexes

In [ ]:
abalone.head()

On calcule la corrélation et on affiche un graphique représentant les différentes relations

In [ ]:
corr = abalone.corr()

corr

In [ ]:
sns.heatmap(corr)

# Construction du modèle

On crée des formules qui nous serviront pour calculer le carré des y actuels et y prédits

In [ ]:
def r_squared(y_actuals, y_predict):
    sst = ((y_actuals - y_actuals.mean()) ** 2).sum()
    sse = ((y_actuals - y_predict) ** 2).sum()
    return 1 - (sse / sst)

In [ ]:
def r_squared_adjusted(y_actuals, y_predict, n_features):
    r_s = r_squared(y_actuals, y_predict)
    n = y_actuals.shape[0]
    return 1 - (1 - r_s) * ((n - 1) / (n - n_features - 1))

On initialise les algorithmes qui nous serviront pour faire nos prédictions, dans ce cas, on utilise le kfold pour
calculer le mean squared error

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
rfecv = RFECV(LinearRegression(), step=1, cv=kfold, scoring='neg_mean_squared_error')

On détermine que le X sera les features à partir duquel on va faire nos calculs, et le Y sera la variable à prédire

In [ ]:
x = abalone[all_features]
y = abalone[name_target]

On impose les résultats obtenus par les valeurs des calculs de X vers le target Y.

In [ ]:
rfecv.fit(x, y)

# Explication du modèle obtenu

À partir des calculés menés dans rfcev.fit, on peut voir quels parties de nos données pourraient être considérees comme
important lors de nos prédictions. Ici on démarre de toutes les features qu'on lui a imposée avec celles déterminées
par rfcev. Puis dans important features on supprimes celles inutiles

In [ ]:
features_and_support = pd.concat([pd.Series(all_features), pd.Series(rfecv.support_)], axis=1)
features_and_support

In [ ]:
important_features = features_and_support[features_and_support.iloc[:, 1]].iloc[:, 0]
important_features

# Prédiction de résultats

On fait nos prédictions sur les valeurs que l'ordinateur a considérées comme utiles

In [ ]:
pred_1 = test[important_features]

lr_1 = LinearRegression()
lr_1.fit(train[important_features], train[name_target])

prediction_1 = lr_1.predict(pred_1)

print("R squared (sklearn):", r2_score(test[name_target], prediction_1))
print("R squared (manual):", r_squared(test[name_target], prediction_1))
print("Adjusted R squared:", r_squared_adjusted(test[name_target], prediction_1, important_features.shape[0]))

In [ ]:
mses = cross_val_score(LinearRegression(), abalone[important_features], abalone[name_target],
                       cv=kfold, scoring='neg_mean_squared_error')
mean_mse = -np.mean(mses)
rmse = mean_mse ** 0.5
print(rmse)
